In [1]:
# define foursquare credentials and version
CLIENT_ID = 'ZDXJNEK0RLOONN2Z1JHMPS3JGUSAGTFWSW4MXURFM2R05B5P' # your Foursquare ID
CLIENT_SECRET = 'KUY4K51J5OZI5WD4WLK2JNS2E5X132VDA1TRA0RQX1KHE2P4' # your Foursquare Secret
VERSION = '20180604'
LIMIT=500

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZDXJNEK0RLOONN2Z1JHMPS3JGUSAGTFWSW4MXURFM2R05B5P
CLIENT_SECRET:KUY4K51J5OZI5WD4WLK2JNS2E5X132VDA1TRA0RQX1KHE2P4


In [4]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
from pandas.io.json import json_normalize 
import requests

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
!conda install -c conda-forge folium=0.5.0 --yes
import folium 
!conda install -c conda-forge lxml --yes
from sklearn.cluster import KMeans

from pandas.io.json import json_normalize 

from sklearn import linear_model
from sklearn.metrics import jaccard_similarity_score


import folium 

from urllib.request import urlopen


import matplotlib.pyplot as plt
import pylab as pl

from sklearn import linear_model
from sklearn.metrics import jaccard_similarity_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, r2_score
import itertools


print('Libraries imported.')


Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    openssl-1.1.1j             |       h7f98852_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  certifi                          2020.12.5-py

In [5]:
#getting coordinates of the address
address1='Ang Mo Kio,Singapore'
location1=Nominatim(user_agent='explorer').geocode(address1)
latitude1=location1.latitude
longitude1=location1.longitude
print("coordinates of {} are {} and {} ".format(address1,latitude1,longitude1))

#getting coordinates of the address
address2='Queenstown,Singapore'
location2=Nominatim(user_agent='explorer').geocode(address2)
latitude2=location2.latitude
longitude2=location2.longitude
print("coordinates of {} are {} and {} ".format(address2,latitude2,longitude2))

#getting coordinates of the address
address3='Tampines,Singapore'
location3=Nominatim(user_agent='explorer').geocode(address3)
latitude3=location3.latitude
longitude3=location3.longitude
print("coordinates of {} are {} and {} ".format(address3,latitude3,longitude3))

coordinates of Ang Mo Kio,Singapore are 1.3700803 and 103.8495228 
coordinates of Queenstown,Singapore are 1.2946226 and 103.8060366 
coordinates of Tampines,Singapore are 1.3546528 and 103.9435712 


In [6]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius

# create URLs
url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude1, 
    longitude1, 
    radius, 
    LIMIT)

url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude2, 
    longitude2, 
    radius, 
    LIMIT)

url3 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude3, 
    longitude3, 
    radius, 
    LIMIT)

In [7]:
# scrape the data from the generated URLs
results1 = requests.get(url1).json()
results1

results2 = requests.get(url2).json()
results2

results3 = requests.get(url3).json()
results3

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
### first city ###    
    
venues1 = results1['response']['groups'][0]['items']
nearby_venues1 = pd.json_normalize(venues1) # flatten JSON

# filter columns
filtered_columns1 = ['venue.name', 'venue.categories', 'venue.location.lat', 
                    'venue.location.lng', 'venue.id']
nearby_venues1 =nearby_venues1.loc[:, filtered_columns1]

# filter the category for each row
nearby_venues1['venue.categories'] = nearby_venues1.apply(get_category_type, axis=1)

# clean columns
nearby_venues1.columns = [col.split(".")[-1] for col in nearby_venues1.columns]

### second city ###

venues2 = results2['response']['groups'][0]['items']
nearby_venues2 =pd.json_normalize(venues2) # flatten JSON

# filter columns
filtered_columns2 = ['venue.name', 'venue.categories', 'venue.location.lat', 
                    'venue.location.lng', 'venue.id']
nearby_venues2 =nearby_venues2.loc[:, filtered_columns2]

# filter the category for each row
nearby_venues2['venue.categories'] = nearby_venues2.apply(get_category_type, axis=1)

# clean columns
nearby_venues2.columns = [col.split(".")[-1] for col in nearby_venues2.columns]

### third city ###

venues3 = results3['response']['groups'][0]['items']
nearby_venues3 = pd.json_normalize(venues3) # flatten JSON

# filter columns
filtered_columns3 = ['venue.name', 'venue.categories', 'venue.location.lat', 
                    'venue.location.lng', 'venue.id']
nearby_venues3 =nearby_venues3.loc[:, filtered_columns3]

# filter the category for each row
nearby_venues3['venue.categories'] = nearby_venues3.apply(get_category_type, axis=1)

# clean columns
nearby_venues3.columns = [col.split(".")[-1] for col in nearby_venues3.columns]

print('{} venues were returned by Foursquare.'.format(nearby_venues1.shape[0]))
print('{} venues were returned by Foursquare.'.format(nearby_venues2.shape[0]))
print('{} venues were returned by Foursquare.'.format(nearby_venues3.shape[0]))

100 venues were returned by Foursquare.
100 venues were returned by Foursquare.
100 venues were returned by Foursquare.


In [8]:
# add locations data to the data sets of each city
nearby_venues1['Location'] = 'Ang Mo Kio'
nearby_venues2['Location'] = 'Queenstown'
nearby_venues3['Location'] = 'Tampines'

In [9]:
# combine the three cities into one data set

nearby_venues = nearby_venues1.copy()
nearby_venues = nearby_venues.append(nearby_venues2)
nearby_venues = nearby_venues.append(nearby_venues3)
nearby_venues.head()

,name,categories,lat,lng,id,Location
0,Bishan - Ang Mo Kio Park,Park,1.362219,103.846250,4b9f4875f964a520e21a37e3,Ang Mo Kio
1,Aramsa ~ The Garden Spa,Spa,1.362292,103.847602,4bea149f61aca5930d6d8300,Ang Mo Kio
2,Central Delights,College Cafeteria,1.377691,103.856590,51219ed9e4b0a73242920174,Ang Mo Kio
3,Bangkok Street Mookata,BBQ Joint,1.365688,103.853186,549a9fe5498e4c9f529e95d9,Ang Mo Kio
4,Old Chang Kee,Snack Place,1.369094,103.848389,4bc30123abf495215220c393,Ang Mo Kio


In [10]:
# check list and manually remove all non-restaurant data
nearby_venues['categories'].unique()

array(['Park', 'Spa', 'College Cafeteria', 'BBQ Joint', 'Snack Place',
       'Supermarket', 'Noodle House', 'Indian Restaurant', 'Burger Joint',
       'Coffee Shop', 'Asian Restaurant', 'Thai Restaurant',
       'Dessert Shop', 'Bakery', 'Seafood Restaurant', 'Trail',
       'Chinese Restaurant', 'Clothing Store', 'Café', 'Reservoir',
       'Diner', 'Ice Cream Shop', 'Breakfast Spot', 'Szechuan Restaurant',
       'Food Court', 'Stadium', 'Historic Site', 'Italian Restaurant',
       'Wine Bar', 'Hotpot Restaurant', 'Bubble Tea Shop',
       'Vegetarian / Vegan Restaurant', 'Office', 'Gym',
       'Nature Preserve', 'Bridge', 'General Entertainment',
       'Sushi Restaurant', 'Non-Profit', 'Cemetery', 'Convenience Store',
       'Flower Shop', 'Pet Store', 'Hakka Restaurant', 'Hotel',
       'Juice Bar', 'Bar', 'Soccer Field', 'Kids Store',
       'Greek Restaurant', 'Furniture / Home Store',
       'Modern European Restaurant', 'Butcher', 'Scenic Lookout',
       'Garden', 'Lounge

In [11]:
removal_list = ['Park','Spa','Supermarket','Mosque','College Cafeteria','Trail','Clothing Store','Reservoir',
                'Stadium','Historic Site','Office','Gym','Nature Preserve','Bridge','General Entertainment','Non-Profit',
                'Farm','Cemetery','Convenience Store','Flower Shop','Pet Store','Soccer Field','Kids Store','Buddhist Temple','Bookstore',
                'Massage Studio','Yoga Studio','Waterfront','Gym / Fitness Center','Shopping Plaza','River','Plaza','Art Gallery',
                'Grocery Store','Pedestrian Plaza','Performing Arts Venue','Outdoor Sculpture','History Museum','Theater','Harbor / Marina','Arcade','Bay','Brewery',
                'Concert Hall','Resort','Museum','Boutique', 'Event Space', 'Church', 'Art Museum', 'Skate Park', 'Furniture / Home Store', 'Gourmet Shop',
                'Pool', 'Ski Chalet', 'Other Great Outdoors', 'College Stadium',
                'High School', 'Kitchen Supply Store', 'Deli / Bodega', 'Hotel'
                'Sporting Goods Shop', 'Golf Course', 'Water Park', 'Airport', 'Airport Lounge', 'Club House', 'Beach Bar', 'Airport Service', 'Beach', 'Garden','Thrift / Vintage Store']
nearby_venues = nearby_venues[~nearby_venues['categories'].isin(removal_list)]

nearby_venues['categories'].unique()

array(['BBQ Joint', 'Snack Place', 'Noodle House', 'Indian Restaurant',
       'Burger Joint', 'Coffee Shop', 'Asian Restaurant',
       'Thai Restaurant', 'Dessert Shop', 'Bakery', 'Seafood Restaurant',
       'Chinese Restaurant', 'Café', 'Diner', 'Ice Cream Shop',
       'Breakfast Spot', 'Szechuan Restaurant', 'Food Court',
       'Italian Restaurant', 'Wine Bar', 'Hotpot Restaurant',
       'Bubble Tea Shop', 'Vegetarian / Vegan Restaurant',
       'Sushi Restaurant', 'Hakka Restaurant', 'Hotel', 'Juice Bar',
       'Bar', 'Greek Restaurant', 'Modern European Restaurant', 'Butcher',
       'Scenic Lookout', 'Lounge', 'Botanical Garden', 'Cocktail Bar',
       'French Restaurant', 'Speakeasy', 'Cupcake Shop', 'Shopping Mall',
       'Salad Place', 'Electronics Store', 'Japanese Restaurant',
       'Pizza Place', 'Dumpling Restaurant', 'Indonesian Restaurant',
       'Movie Theater', 'Cosmetics Shop', 'Bistro', 'Tea Room',
       'Fried Chicken Joint', 'Sporting Goods Shop',
       

In [12]:
# set up to pull the likes from the API based on venue ID
url_list = []
like_list = []
json_list = []

for i in list(nearby_venues.id):
    venue_url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(i, CLIENT_ID, CLIENT_SECRET, VERSION)
    url_list.append(venue_url)
for link in url_list:
    result = requests.get(link).json()
    likes = result['response']['likes']['count']
    like_list.append(likes)
print(like_list)


nearby_venues['likes'] = like_list
nearby_venues.head()

[15, 5, 10, 81, 33, 7, 6, 17, 63, 11, 142, 55, 15, 29, 20, 7, 28, 7, 287, 64, 17, 9, 81, 14, 9, 43, 18, 91, 6, 11, 16, 18, 4, 13, 8, 21, 135, 3, 36, 47, 37, 60, 26, 9, 29, 105, 11, 178, 25, 35, 9, 30, 98, 9, 29, 21, 14, 35, 89, 149, 21, 6, 19, 10, 6, 5, 93, 245, 33, 182, 53, 137, 394, 95, 42, 263, 177, 11, 1221, 19, 152, 126, 311, 9, 49, 10, 173, 27, 24, 343, 26, 117, 56, 140, 3701, 20, 17, 22, 268, 71, 320, 75, 21, 469, 57, 55, 408, 37, 12, 480, 106, 577, 2277, 27, 19, 64, 1386, 35, 16, 34, 214, 38, 225, 22, 95, 30, 105, 36, 159, 34, 19, 124, 27, 3157, 583, 9, 10, 11, 14, 14, 27, 31, 84, 117, 83, 5, 8, 12, 144, 14, 17, 8, 13, 12, 18, 107, 277, 53, 17, 10, 20, 9, 6, 3, 32, 136, 21, 69, 21, 12, 44, 37, 10, 39, 96, 9, 23, 15, 59, 23, 7, 25, 19, 7, 9, 6, 43, 59, 108, 5, 4, 166, 18, 69, 30, 191, 6, 10, 13]


,name,categories,lat,lng,id,Location,likes
3,Bangkok Street Mookata,BBQ Joint,1.365688,103.853186,549a9fe5498e4c9f529e95d9,Ang Mo Kio,15
4,Old Chang Kee,Snack Place,1.369094,103.848389,4bc30123abf495215220c393,Ang Mo Kio,5
7,Face Ban Mian 非板面 (Ang Mo Kio),Noodle House,1.372031,103.847504,50712b67e4b0548504446dea,Ang Mo Kio,10
8,Sin Ming Roti Prata (Faisal & Aziz Curry Musli...,Indian Restaurant,1.355269,103.836719,4b584116f964a520c14f28e3,Ang Mo Kio,81
9,MOS Burger,Burger Joint,1.369170,103.847831,4b1ea04ff964a520e21c24e3,Ang Mo Kio,33


In [13]:
# this is really the raw dataset now so let us rename it something more appropriate

raw_dataset = nearby_venues
raw_dataset.head()

,name,categories,lat,lng,id,Location,likes
3,Bangkok Street Mookata,BBQ Joint,1.365688,103.853186,549a9fe5498e4c9f529e95d9,Ang Mo Kio,15
4,Old Chang Kee,Snack Place,1.369094,103.848389,4bc30123abf495215220c393,Ang Mo Kio,5
7,Face Ban Mian 非板面 (Ang Mo Kio),Noodle House,1.372031,103.847504,50712b67e4b0548504446dea,Ang Mo Kio,10
8,Sin Ming Roti Prata (Faisal & Aziz Curry Musli...,Indian Restaurant,1.355269,103.836719,4b584116f964a520c14f28e3,Ang Mo Kio,81
9,MOS Burger,Burger Joint,1.369170,103.847831,4b1ea04ff964a520e21c24e3,Ang Mo Kio,33


In [14]:
# inspecting the raw dataset shows that there may be too many different types of cuisines
raw_dataset['categories'].unique()

euro = ['Pizza Place','Italian Restaurant','French Restaurant','Persian Restaurant','Spanish Restaurant', 'Greek Restaurant', 'Modern European Restaurant','Scandinavian Restaurant']

bar = ['Cocktail Bar', 'Wine Bar', 'Hotel Bar',
       'Beer Garden', 'Pub', 'Bar', 'Gastropub', 'Juice Bar']

asian = ['Indian Restaurant','Korean Restaurant','Chinese Restaurant',
       'Thai Restaurant', 'Noodle House', 'Szechuan Restaurant'
       'Asian Restaurant', 'Hakka Restaurant',  'Hotpot Restaurant'
       'Japanese Restaurant','Afghan Restaurant',
       'Vietnamese Restaurant', 'Sushi Restaurant', 'Dumpling Restaurant', 'Filipino Restaurant']

casual = ['Snack Place',
       'Dessert Shop','Fast Food Restaurant','Café','Coffee Shop','BBQ Joint', 'Food Court', 'Bubble Tea Shop', 
       'Deli / Bodega','Diner','Sandwich Place','Salad Place','Bagel Shop','Vegetarian / Vegan Restaurant', 'Tea Room','Cupcake Shop', 'Street Food Gathering'
       'Bakery', 'Ice Cream Shop', 'Breakfast Spot','Burger Joint','Bubble Tea Shop',
       'Fried Chicken Joint']

american = ['Steakhouse', 'Restaurant', 'Seafood Restaurant']

def conditions(s):
    if s['categories'] in euro:
        return 'euro'
    if s['categories'] in asian:
        return 'asian'
    if s['categories'] in casual:
        return 'casual'
    if s['categories'] in american:
        return 'american'
    if s['categories'] in bar:
        return 'bar'


raw_dataset['categories_classified']=raw_dataset.apply(conditions, axis=1)
raw_dataset.head()


# double check to make sure categories_classified has been created correctly
pd.crosstab(index=raw_dataset["categories_classified"],
            columns="count")

col_0,count
categories_classified,
american,7
asian,39
bar,6
casual,76
euro,12


In [15]:
# classify the likes into different ranking levels
print(np.percentile(raw_dataset['likes'], 33))
print(np.percentile(raw_dataset['likes'], 66))

17.0
55.68000000000001


In [16]:
def rankings(s):
    if s['likes']<=13:
        return 1
    if s['likes']<=35:
        return 2
    if s['likes']>35:
        return 3

raw_dataset['ranking']=raw_dataset.apply(rankings, axis=1)
raw_dataset.head()

,name,categories,lat,lng,id,Location,likes,categories_classified,ranking
3,Bangkok Street Mookata,BBQ Joint,1.365688,103.853186,549a9fe5498e4c9f529e95d9,Ang Mo Kio,15,casual,2
4,Old Chang Kee,Snack Place,1.369094,103.848389,4bc30123abf495215220c393,Ang Mo Kio,5,casual,1
7,Face Ban Mian 非板面 (Ang Mo Kio),Noodle House,1.372031,103.847504,50712b67e4b0548504446dea,Ang Mo Kio,10,asian,1
8,Sin Ming Roti Prata (Faisal & Aziz Curry Musli...,Indian Restaurant,1.355269,103.836719,4b584116f964a520c14f28e3,Ang Mo Kio,81,asian,3
9,MOS Burger,Burger Joint,1.369170,103.847831,4b1ea04ff964a520e21c24e3,Ang Mo Kio,33,casual,2


In [17]:
# create dummies for linear regression modelling

# one hot encoding
reg_dataset = pd.get_dummies(raw_dataset[['categories_classified', 
                                          'Location',]], 
                               prefix="", 
                               prefix_sep="")

# add name, ranking, and likes columns back to dataframe
reg_dataset['ranking'] = raw_dataset['ranking']
reg_dataset['likes'] = raw_dataset['likes']
reg_dataset['name'] = raw_dataset['name']

# move name column to the first column
reg_columns = [reg_dataset.columns[-1]] + list(reg_dataset.columns[:-1])
reg_dataset = reg_dataset[reg_columns]
reg_dataset.head()

# Training and Testing
y_data=reg_dataset['likes']
x_data=reg_dataset.drop('likes',axis=1)
msk = np.random.rand(len(reg_dataset)) < 0.8
train = reg_dataset[msk]
test = reg_dataset[~msk]

x_train = np.asanyarray(train[['american', 'asian', 'bar', 'casual','euro','Ang Mo Kio','Queenstown','Tampines']])
y_train = np.asanyarray(train[['likes']])

x_test = np.asanyarray(test[['american', 'asian', 'bar', 'casual','euro','Ang Mo Kio','Queenstown','Tampines']])

y_test = np.asanyarray(test[['likes']])

print("number of test samples :", x_test.shape[0])
print("number of training samples:",x_train.shape[0])

# Multinomial Ordinal Logistic Regression

x_train = np.asanyarray(train[['american', 'asian', 'bar', 'casual','euro','Ang Mo Kio','Queenstown','Tampines']])
y_train = np.asanyarray(train['ranking'])

x_test = np.asanyarray(test[['american', 'asian', 'bar', 'casual','euro','Ang Mo Kio','Queenstown','Tampines']])
y_test = np.asanyarray(test['ranking'])


# LR = LogisticRegression(C=0.01, solver='liblinear').fit(x_train, y_train)
# LR

mul_ordinal = linear_model.LogisticRegression(multi_class='multinomial',
                                              solver='newton-cg',
                                              fit_intercept=True).fit(x_train,
                                                                      y_train)

mul_ordinal

coef = mul_ordinal.coef_[0]
print (coef)

number of test samples : 37
number of training samples: 162
[ 0.06365263 -0.42559712  0.32686363 -0.20169922 -0.78435199  0.31060716
 -0.72784385  0.41723849]


In [18]:
# Multinomial Ordinal Logistic Regression Prediction Capabilities

yhat_test = mul_ordinal.predict(x_test)
yhat_test

yhat_testprob = mul_ordinal.predict_proba(x_test)
yhat_testprob

jaccard_similarity_score(y_test, yhat_test)

0.5405405405405406

In [19]:
from sklearn.metrics import log_loss
log_loss(y_test, yhat_testprob)

0.977678659744474